In [20]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):  # Changed _init_ to __init__
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None  # Initialize __driver to None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))  # Use self.__uri, self.__user, self.__pwd and assign to self.__driver
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() # Use self.__driver
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

    def get_driver(self):
        return self.__driver #Return self.__driver

In [11]:
# LLM Initialization (Ollama)
# -----------------------------------
llm = Ollama(model="llama2", verbose=True, temperature=0.

SyntaxError: incomplete input (3124644094.py, line 3)

In [19]:
def populate_neo4j(conn, data):
    # Access the driver using the getter method
    with conn.get_driver().session() as session:  # Use a transaction
        for item in tqdm(data, total=len(data)):  # Iterate through the list of dictionaries
            wall_id = item["wall_id"]
            materials = item["materials"]

            # Create Wall node
            session.run("""
                MERGE (w:Wall {id: $wall_id})
            """, parameters={'wall_id': wall_id})

            # Create Material nodes and relationships
            for material in materials:
                session.run("""
                    MERGE (m:Material {name: $name})
                    ON CREATE SET m.density = $density, m.conductivity = $conductivity,
                                 m.u_value = $u_value, m.embodied_carbon_coefficient = $carbon_coeff,
                                 m.cost = $cost, m.recyclability = $recyclability,
                                 m.bio_based = $bio_based, m.color = $color
                    ON MATCH SET m.density = $density, m.conductivity = $conductivity,
                                 m.u_value = $u_value, m.embodied_carbon_coefficient = $carbon_coeff,
                                 m.cost = $cost, m.recyclability = $recyclability,
                                 m.bio_based = $bio_based, m.color = $color
                    MERGE (w:Wall {id: $wall_id})
                    MERGE (m)-[:USED_IN {thickness: $thickness}]->(w)
                """, parameters={
                    'name': material['material'],
                    'wall_id': wall_id,
                    'thickness': material['thickness'],
                    'density': material['density'],
                    'conductivity': material['conductivity'],
                    'u_value': material['u_value'],
                    'carbon_coeff': material['embodied_carbon_coefficient'],
                    'cost': material['cost'],
                    'recyclability': material['recyclability'],
                    'bio_based': material['bio_based'],
                    'color': material['color'],
                    'score': overall_score
                })

            # Create Metric nodes for additional properties and relationships
            metrics = {
                'construction_demolition_waste': item.get('construction_demolition_waste'),
                'circular_economy': item.get('circular_economy'),
                'heritage_preservation': item.get('heritage_preservation'),
                'responsible_material_sourcing': item.get('responsible_material_sourcing'),
                'embodied_ghg_emissions': item.get('embodied_ghg_emissions'),
                'affordable_adoption_high_quality_housing_conditions': item.get('affordable_adoption_high_quality_housing_conditions')
            }

            # Loop over each metric and create nodes and relationships
            for metric_name, metric_value in metrics.items():
                if metric_value is not None:
                    session.run("""
                        MERGE (k:Metric {name: $metric_name})
                        ON CREATE SET k.value = $value
                        ON MATCH SET k.value = $value
                        MERGE (w:Wall {id: $wall_id})
                        MERGE (w)-[:HAS_METRIC {value: $value}]->(m)
                    """, parameters={
                        'metric_name': metric_name,
                        'value': metric_value,
                        'Tagline': tagline,
                        
                    })

In [2]:
import json
from neo4j import GraphDatabase
from tqdm import tqdm

class Neo4jConnection:
    def __init__(self, uri, user, password):
        # Create a driver instance
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        if self._driver is not None:
            self._driver.close()
    
    def get_driver(self):
        return self._driver

def populate_neo4j(conn, data):
    # Access the driver using the getter method
    with conn.get_driver().session() as session:  # Use a transaction
        for item in tqdm(data, total=len(data)):  # Iterate through the list of dictionaries
            wall_id = item["wall_id"]
            materials = item["materials"]

            # Create Wall node
            session.run("""
                MERGE (w:Wall {id: $wall_id})
            """, parameters={'wall_id': wall_id})

            # Create Material nodes and relationships
            for material in materials:
                session.run("""
                    MERGE (m:Material {name: $name})
                    ON CREATE SET m.density = $density, m.conductivity = $conductivity,
                                 m.u_value = $u_value, m.embodied_carbon_coefficient = $carbon_coeff,
                                 m.cost = $cost, m.recyclability = $recyclability,
                                 m.bio_based = $bio_based, m.color = $color
                    ON MATCH SET m.density = $density, m.conductivity = $conductivity,
                                 m.u_value = $u_value, m.embodied_carbon_coefficient = $carbon_coeff,
                                 m.cost = $cost, m.recyclability = $recyclability,
                                 m.bio_based = $bio_based, m.color = $color
                    MERGE (w:Wall {id: $wall_id})
                    MERGE (m)-[:USED_IN {thickness: $thickness}]->(w)
                """, parameters={
                    'name': material['material'],
                    'wall_id': wall_id,
                    'thickness': material['thickness'],
                    'density': material['density'],
                    'conductivity': material['conductivity'],
                    'u_value': material['u_value'],
                    'carbon_coeff': material['embodied_carbon_coefficient'],
                    'cost': material['cost'],
                    'recyclability': material['recyclability'],
                    'bio_based': material['bio_based'],
                    'color': material['color'],
                })

            # Create Metric nodes for additional properties and relationships, including overall_score
            metrics = {
                'construction_demolition_waste': item.get('construction_demolition_waste'),
                'circular_economy': item.get('circular_economy'),
                'heritage_preservation': item.get('heritage_preservation'),
                'responsible_material_sourcing': item.get('responsible_material_sourcing'),
                'embodied_ghg_emissions': item.get('embodied_ghg_emissions'),
                'affordable_adoption_high_quality_housing_conditions': item.get('affordable_adoption_high_quality_housing_conditions'),
                'overall_score': item.get('overall_score')
            }

            # Loop over each metric and create nodes and relationships
            for metric_name, metric_value in metrics.items():
                if metric_value is not None:
                    session.run("""
                        MERGE (m:Metric {name: $metric_name})
                        ON CREATE SET m.value = $value
                        ON MATCH SET m.value = $value
                        MERGE (w:Wall {id: $wall_id})
                        MERGE (w)-[:HAS_METRIC {value: $value}]->(m)
                    """, parameters={
                        'metric_name': metric_name,
                        'value': metric_value,
                        'wall_id': wall_id
                    })

if __name__ == "__main__":
    # Path to your JSON file
    file_path = 'wall_population.json'  # Adjust the path if needed

    # Load JSON data
    print("Loading data...")
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Neo4j connection details
    uri = "neo4j+s://1335f3b1.databases.neo4j.io"  # Replace with your Neo4j URI
    user = "neo4j"  # Replace with your Neo4j username
    password = "o72Bks0bw34x0rRFvwKHKs82KZoA_JuJn-jjYqR5QII"  # Replace with your Neo4j password

    # Establish connection
    print("Connecting to Neo4j...")
    conn = Neo4jConnection(uri, user, password)

    # Test the connection by running a simple query
    try:
        with conn.get_driver().session() as session:
            result = session.run("RETURN 1 AS test")
            print("Test query result:", result.single())
    except Exception as e:
        print("Failed to establish Neo4j connection:", e)
        conn.close()
        exit(1)

    # Populate Neo4j database
    print("Populating the Neo4j database...")
    try:
        populate_neo4j(conn, data)
    except Exception as e:
        print("An error occurred while populating the database:", e)
    finally:
        # Close connection
        conn.close()
        print("Neo4j database population complete.")


Loading data...
Connecting to Neo4j...
Test query result: <Record test=1>
Populating the Neo4j database...


100%|██████████| 2000/2000 [13:51<00:00,  2.41it/s]

Neo4j database population complete.


In [9]:
def print_schema(conn):
    with conn.get_driver().session() as session:
        # Retrieve a high-level schema visualization (node labels and relationship types)
        schema_vis = session.run("CALL db.schema.visualization()")
        schema_data = schema_vis.data()
        print("Schema Visualization:")
        print(schema_data)

        # Retrieve node label and property information (Neo4j 5+):
        node_props = session.run("CALL db.schema.nodeTypeProperties()")
        node_props_data = node_props.data()
        print("\nNode Type Properties:")
        for record in node_props_data:
            print(record)

        # Retrieve relationship type and property information (Neo4j 5+):
        rel_props = session.run("CALL db.schema.relationshipTypeProperties()")
        rel_props_data = rel_props.data()
        print("\nRelationship Type Properties:")
        for record in rel_props_data:
            print(record)


if __name__ == "__main__":
    # Path to your JSON file
    file_path = 'wall_population.json'  # Adjust path if needed

    # Load JSON data
    print("Loading data...")
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Neo4j connection details
    uri = "neo4j+s://1335f3b1.databases.neo4j.io"  # Your Neo4j URI
    user = "neo4j"  # Your username
    password = "o72Bks0bw34x0rRFvwKHKs82KZoA_JuJn-jjYqR5QII"  # Your password

    # Establish connection
    print("Connecting to Neo4j...")
    conn = Neo4jConnection(uri, user, password)

    # Test the connection by running a simple query
    try:
        with conn.get_driver().session() as session:
            result = session.run("RETURN 1 AS test")
            print("Test query result:", result.single())
    except Exception as e:
        print("Failed to establish Neo4j connection:", e)
        conn.close()
        exit(1)

    # Populate Neo4j database
    print("Populating the Neo4j database...")
    try:
        populate_neo4j(conn, data)
    except Exception as e:
        print("An error occurred while populating the database:", e)
    finally:
        # Print schema information after population
        print("\nRetrieving and printing schema information:")
        print_schema(conn)

        # Close connection
        conn.close()
        print("Neo4j database population complete.")


Loading data...
Connecting to Neo4j...
Test query result: <Record test=1>
Populating the Neo4j database...


100%|██████████| 2000/2000 [13:20<00:00,  2.50it/s]



Retrieving and printing schema information:
Schema Visualization:
[{'nodes': [{'name': 'Wall', 'indexes': [], 'constraints': []}, {'name': 'Material', 'indexes': [], 'constraints': []}, {'name': 'Metric', 'indexes': [], 'constraints': []}], 'relationships': [({'name': 'Material', 'indexes': [], 'constraints': []}, 'USED_IN', {'name': 'Wall', 'indexes': [], 'constraints': []}), ({'name': 'Wall', 'indexes': [], 'constraints': []}, 'HAS_METRIC', {'name': 'Metric', 'indexes': [], 'constraints': []})]}]

Node Type Properties:
{'nodeType': ':`Material`', 'nodeLabels': ['Material'], 'propertyName': 'name', 'propertyTypes': ['String'], 'mandatory': True}
{'nodeType': ':`Material`', 'nodeLabels': ['Material'], 'propertyName': 'density', 'propertyTypes': ['Double'], 'mandatory': True}
{'nodeType': ':`Material`', 'nodeLabels': ['Material'], 'propertyName': 'conductivity', 'propertyTypes': ['Double'], 'mandatory': True}
{'nodeType': ':`Material`', 'nodeLabels': ['Material'], 'propertyName': 'u_va

ClientError: {code: Neo.ClientError.Procedure.ProcedureNotFound} {message: There is no procedure with the name `db.schema.relationshipTypeProperties` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.}

In [18]:
cypher = """
  MATCH (n) 
  RETURN count(n)
  """
result = graph.query(cypher)
result

NameError: name 'graph' is not defined

In [ ]:
cypher = """
  MATCH  (m:Material) 
  RETURN count(m) AS numberOfMaterial 
  """
graph.query(cypher)


[{'numberOfMaterial': 18}]

In [ ]:
cypher = """
  MATCH (rockwool:Material {name:"rockwool"}) 
  RETURN rockwool
  """
graph.query(cypher)

[]

In [ ]:
cypher_query = """
MATCH (w:Wall {id: $wall_id})-[:HAS_METRIC]->(m:Metric)
RETURN m.name AS metric_name, m.value AS metric_value
"""

parameters = {'wall_id': 1010}  # Replace with the actual wall ID
results = graph.query(cypher_query, parameters)

print(results)

[{'metric_name': 'construction_demolition_waste', 'metric_value': 68.46}, {'metric_name': 'circular_economy', 'metric_value': 91.29}, {'metric_name': 'heritage_preservation', 'metric_value': 0.0}, {'metric_name': 'responsible_material_sourcing', 'metric_value': 100.0}, {'metric_name': 'embodied_ghg_emissions', 'metric_value': 93.69}]


In [ ]:
from datasets import load_dataset, Dataset

# Load your custom JSON dataset
data = [
    {"natural_query": "Find all nodes connected to node A.", "cypher_query": "MATCH (a {name: 'A'})--(b) RETURN b;"},
    {"natural_query": "Get the shortest path between node A and node B.", "cypher_query": "MATCH p=shortestPath((a {name: 'A'})-[*]-(b {name: 'B'})) RETURN p;"}
]

dataset = Dataset.from_list(data)

# Split into train and validation sets
dataset = dataset.train_test_split(test_size=0.2)
print(dataset)


c:\Users\Lenovo\.conda\envs\First_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['natural_query', 'cypher_query'],
        num_rows: 1
    })
    test: Dataset({
        features: ['natural_query', 'cypher_query'],
        num_rows: 1
    })
})


In [ ]:
q_one = "What are the materials in wall 1?"
q_two = "What is the cricluar economic score for wall 1?"

In [ ]:

'''
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_core.tools import Tool # Make sure to import Tool 
# Import the create_react_agent and AgentExecutor functions 
from langchain.agents import AgentExecutor, create_react_agent,AgentOutputParser  # Make sure you import the hub object for pulling prompts 
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph
from uuid import uuid4
import streamlit as st
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = 'True'
os.environ["LANGCHAIN_API_KEY"] = str(os.getenv("LANGCHAIN_API_KEY"))

# Neo4j Connection Class
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        if self._driver:
            self._driver.close()

    def query(self, query, parameters=None):
        with self._driver.session() as session:
            return session.run(query, parameters).data()
        
'''
# Define entities specific to this dataset
class WallEntities(BaseModel):
    """Identifying information about walls and materials."""
    materials: List[str] = Field(
        ..., description="List of material names mentioned in the text."
    )
    wall_ids: List[int] = Field(
        ..., description="List of wall IDs mentioned in the text."
    )
    properties: List[int] = Field(
        ..., description="List of properties (e.g., recyclability, embodied carbon) mentioned."
    )
    
# Define the prompt template for conversation
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You extract materials, wall IDs, and properties from user queries."),
        ("human", "Extract entities from this query: What are the materials in wall 1?"),
        
    ]
)
'''
def query_graph_with_llm(session, query):
    graph_data = fetch_graph_data(session)  # Step 1: Fetch data
def format_graph_data_to_nlp(graph_data):
    description = "Here is the data from the Neo4j graph:\n\n"
    for record in graph_data:
        material = record["material"]
        wall = record["wall"]
        relationship = record["relationship"]
        metrics = record["metrics"]

        description += f"Material '{material['name']}' has the following properties:\n"
        description += f"  - Density: {material['density']} kg/m³\n"
        description += f"  - Conductivity: {material['conductivity']} W/mK\n"
        description += f"  - U-Value: {material['u_value']}\n"
        description += f"  - Recyclability: {material['recyclability']} out of 5\n"
        description += f"  - Bio-Based: {'Yes' if material['bio_based'] else 'No'}\n"

        description += f"It is used in Wall ID {wall['id']} with a thickness of {relationship['thickness']} m.\n"

        if metrics:
            description += "The wall has the following metrics:\n"
            for metric in metrics:
                description += f"  - {metric['name']}: {metric['value']}\n"
        description += "\n"

    return description
# Define the prompt template for conversation
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You extract materials, wall IDs, and properties from user queries."),
        ("human", "Extract entities from this query: What are the walls with bio based material in it?"),
        
    ]
)
'''
  # Format as natural language (or JSON)
    for record in graph_data:
        material = record['material']
        wall = record['wall']
        relationship = record['relationship']
        metrics = record['metrics']

        description += f"Material: {material['name']} used in Wall ID: {wall['id']}.\n"
        if metrics:
            for metric in metrics:
                description += f"  - {metric['name']}: {metric['value']}\n"
        description += "\n"

    # Step 2: Pass to LLM
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You analyze construction data from the graph."),
        ("human", f"Here is the graph data:\n{description}\n\nQuestion: {query}")
    ])
    llm_response = llm.run({"query": query})
    return llm_response
'''
# Set up Neo4j Graph connection
graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="123456789"
)

# Questions
q_one = "What are the materials in with bio based material in it?"


# Ollama LLaMA2 LLM
llm = Ollama(model="llama2")

# Create Cypher chat function using StrOutputParser
output_parser = StrOutputParser()
cypher_chat = prompt | llm | output_parser

# Streamlit app setup
st.title("Neo4j and LangChain Interaction")

# Get user input from a text box
input_text = st.text_input("Enter your question or command:")

# Streamlit input handling (if used with Streamlit)
if input_text:  # Assuming `input_text` is a user input from Streamlit
    st.write(cypher_chat.invoke({"input": input_text}))  # Ensure "input" matches the template

# Response for q_one
response_one = cypher_chat.invoke({"input": q_one})


# Print results
print("Response for Q1:", response_one)

print("\nResponse for Q2:", response_two)

print("Prompt Template:")
print(prompt)
'''





2024-12-15 23:27:19.112 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-15 23:27:19.127 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-15 23:27:19.135 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-15 23:27:19.141 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-15 23:27:19.146 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-15 23:27:19.147 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-15 23:27:19.169 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-15 23:27:19.171 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Response for Q1: 
Sure! Based on your query, I can extract the following entities:

1. Materials: "bio"

Response for Q2: 
Entities:

* Materials: Not specified
* Wall ID: 1
* Property: Not specified
Prompt Template:
input_variables=[] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You extract materials, wall IDs, and properties from user queries.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Extract entities from this query: What are the walls with bio based material in it?'), additional_kwargs={})]


In [53]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_core.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent, AgentOutputParser
from langchain import hub
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph
from uuid import uuid4
import streamlit as st
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

# Load environment variables
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = 'True'
os.environ["LANGCHAIN_API_KEY"] = str(os.getenv("LANGCHAIN_API_KEY"))

# Neo4j Connection Class
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        if self._driver:
            self._driver.close()

    def query(self, query, parameters=None):
        with self._driver.session() as session:
            return session.run(query, parameters).data()

# Example function to fetch graph data
def fetch_graph_data(session):
    return session.query("MATCH (n) RETURN n LIMIT 25")

def format_graph_data_to_nlp(graph_data):
    description = "Here is the data from the Neo4j graph:\n\n"
    for record in graph_data:
        material = record["material"]
        wall = record["wall"]
        relationship = record["relationship"]
        metrics = record["metrics"]

        description += f"Material '{material['name']}' has the following properties:\n"
        description += f"  - Density: {material['density']} kg/m³\n"
        description += f"  - Conductivity: {material['conductivity']} W/mK\n"
        description += f"  - U-Value: {material['u_value']}\n"
        description += f"  - Recyclability: {material['recyclability']} out of 5\n"
        description += f"  - Bio-Based: {'Yes' if material['bio_based'] else 'No'}\n"

        description += f"It is used in Wall ID {wall['id']} with a thickness of {relationship['thickness']} m.\n"

        if metrics:
            description += "The wall has the following metrics:\n"
            for metric in metrics:
                description += f"  - {metric['name']}: {metric['value']}\n"
        description += "\n"

    return description

# Define the prompt template for conversation

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an assistant who understands the relationships and nodes in a Neo4j graph database. You help by interpreting and responding to queries about the data in this database."),
        ("human", "What are the materials used in 1021 in the database?"),
    ]
)

# Set up Neo4j Graph connection
graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="123456789"
)

# Adjusted Cypher query
cypher_query = """
MATCH (m:Material) 
WHERE m.bio_based = true 
RETURN m.name AS name, m.density AS density, m.conductivity AS conductivity, m.u_value AS u_value, m.recyclability AS recyclability
"""

# Ollama LLaMA2 LLM
llm = Ollama(model="llama2", verbose=True, temperature=0.7)

# Create Cypher chat function using StrOutputParser
output_parser = StrOutputParser()
cypher_chat = prompt | llm | output_parser

# Streamlit app setup
st.title("Neo4j and LangChain Interaction")

# Get user input from a text box
input_text = st.text_input("Enter your question or command:")

# Streamlit input handling (if used with Streamlit)
if input_text:  # Assuming `input_text` is a user input from Streamlit
    st.write(cypher_chat.invoke({"input": input_text}))  # Ensure "input" matches the template

# Response for the Cypher query
response_one = cypher_chat.invoke({"input": cypher_query})

# Print results
print("Response for Cypher Query:", response_one)

print("Prompt Template:")
print(prompt)


2024-12-16 20:08:01.144 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 20:08:01.144 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 20:08:01.144 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 20:08:01.144 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 20:08:01.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 20:08:01.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 20:08:01.155 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 20:08:01.155 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Response for Cypher Query: 
As an assistant, I can access and interpret the data stored in the Neo4j graph database. Based on your query, I can confirm that there are several materials associated with the node labeled "1021" in the database.

Here is a list of the materials used by 1021, as retrieved from the database:

1. Aluminum - This material is connected to 1021 through a relationship labeled "uses."
2. Carbon fiber - This material is also connected to 1021 through the "uses" relationship.
3. Copper - There is an edge labeled "produces" connecting 1021 to copper.
4. Glass - Glass is connected to 1021 through a relationship labeled "manufactures."
5. Kevlar - Kevlar is associated with 1021 through the "uses" relationship.
6. Plastic - There is an edge labeled "produces" connecting 1021 to plastic.
7. Steel - Steel is connected to 1021 through a relationship labeled "manufactures."

I hope this information helps answer your query! Let me know if you have any further questions or if

Response for Cypher Query: We are getting from our llm
As an assistant, I can access and interpret the data stored in the Neo4j graph database. Based on your query, I can confirm that there are several materials associated with the node labeled "1021" in the database.

Here is a list of the materials used by 1021, as retrieved from the database:

1. Aluminum - This material is connected to 1021 through a relationship labeled "uses."
2. Carbon fiber - This material is also connected to 1021 through the "uses" relationship.
3. Copper - There is an edge labeled "produces" connecting 1021 to copper.
4. Glass - Glass is connected to 1021 through a relationship labeled "manufactures."
5. Kevlar - Kevlar is associated with 1021 through the "uses" relationship.
6. Plastic - There is an edge labeled "produces" connecting 1021 to plastic.
7. Steel - Steel is connected to 1021 through a relationship labeled "manufactures."

I hope this information helps answer your query! Let me know if you have any further questions or if there's anything else I can help with.
Prompt Template:
input_variables=[] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an assistant who understands the relationships and nodes in a Neo4j graph database. You help by interpreting and responding to queries about the data in this database.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='What are the materials used in 1021 in the database?'), additional_kwargs={})]


In [ ]:
import os
from typing import List
from langchain.chains.openai_functions import create_structured_output_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.llms import Ollama

# Enable LangChain tracing and set API key
os.environ["LANGCHAIN_TRACING_V2"] = 'True'
os.environ["LANGCHAIN_API_KEY"] = str(os.getenv("LANGCHAIN_API_KEY"))

# Initialize the LLM
llm = Ollama(model="llama2")

# Define entities specific to this dataset
class WallEntities(BaseModel):
    """Identifying information about walls and materials."""
    materials: List[str] = Field(
        ..., description="List of material names mentioned in the text."
    )
    wall_ids: List[int] = Field(
        ..., description="List of wall IDs mentioned in the text."
    )
    properties: List[str] = Field(
        ..., description="List of properties (e.g., recyclability, embodied carbon) mentioned."
    )

class WallDataPipeline:
    def __init__(self, llm, data):
        self.llm = llm
        self.data = data  # JSON data loaded into memory

    # Step 1: Entity extraction
    def prepare_entity_chain(self):
        entity_chain_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "You extract materials, wall IDs, and properties from user queries."),
                ("human", "Extract entities from this query: {query}"),
            ]
        )
        entity_chain = create_structured_output_chain(WallEntities, self.llm, entity_chain_prompt)
        return entity_chain

    # Step 2: Map entities to the dataset
    def map_to_data(self, entities: WallEntities):
        """Map extracted entities to the JSON dataset."""
        results = []

        # Find walls by ID
        for wall_id in entities.wall_ids:
            wall = next((w for w in self.data if w["wall_id"] == wall_id), None)
            if wall:
                results.append({"wall_id": wall_id, "data": wall})

        # Find materials
        for material in entities.materials:
            for wall in self.data:
                for mat in wall["materials"]:
                    if material.lower() in mat["material"].lower():
                        results.append({"wall_id": wall["wall_id"], "material": mat})

        # Find walls by properties
        for property_name in entities.properties:
            for wall in self.data:
                if property_name in wall:
                    results.append({"wall_id": wall["wall_id"], "property": {property_name: wall[property_name]}})

        return results

    # Step 3: Generate responses
    def prepare_response_prompt(self):
        response_template = """Based on the question, generate a response using the matched data:
        Question: {query}
        Matched Data: {data}"""
        response_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "Generate a natural language response."),
                ("human", response_template),
            ]
        )
        return response_prompt

    def prepare_pipeline(self):
        entity_chain = self.prepare_entity_chain()
        response_prompt = self.prepare_response_prompt()

        pipeline = (
            RunnablePassthrough.assign(entities=entity_chain)
            | RunnablePassthrough.assign(matched_data=lambda x: self.map_to_data(x["entities"]))
            | response_prompt
            | self.llm
            | StrOutputParser()
        )

        return pipeline

    def run_pipeline(self, query):
        entity_chain = self.prepare_entity_chain()
        entities = entity_chain.run({"query": query})
        matched_data = self.map_to_data(entities)
        response_prompt = self.prepare_response_prompt()
        response = response_prompt.run({"query": query, "data": matched_data})
        return response
verbose = True
